In [32]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [33]:
#get all the data into one big frame
file = "../data/v3moves.csv"
wholeFrame = pd.read_csv(file)

In [34]:
#view the first 5 lines of our initial frame
wholeFrame.head()

,board,from,to,promote
0,0000000000000000000000000000000000010000101000...,44,35,x
1,0000000000000000000000000000100000000000101000...,35,28,x
2,0000000000000000000000000000100000000000101000...,46,28,x
3,0000000000000010000000000000000000000000101000...,28,14,x
4,0000000000000010000000000000000000000000101000...,62,37,x


In [35]:
#build the frames we want
moveFrame = wholeFrame.drop(['promote'], axis=1)
promoteFrame = wholeFrame.drop(['from', 'to'], axis=1)

In [36]:
#remove duplicates
moveFrame = moveFrame.drop_duplicates()
promoteFrame = promoteFrame.drop_duplicates()

#remove rows where theres no promotion
promoteFrame = promoteFrame[promoteFrame.promote != "x"]

In [37]:
#update indices because we've removed some stuff
moveFrame = moveFrame.reset_index()
promoteFrame = promoteFrame.reset_index()

#remove the old indices
moveFrame = moveFrame.drop(['index'], axis=1)
promoteFrame = promoteFrame.drop(['index'], axis=1)

In [38]:
#shuffle frames
moveFrame = moveFrame.sample(frac = 1)
promoteFrame = promoteFrame.sample(frac = 1)

In [39]:
#function for spliting the board into individual chars
def parseBoard(board):
    return np.asarray([char for char in board]).astype("float32")

In [40]:
#function for converting 'to' and 'from' to formats the nn can use
#instead of storing 10 it stores [0,0,0,0,0,0,0,0,0,1,0,0,0,0,...] for example
def parseMove(move):
    result = np.zeros(64,dtype = np.float32)
    result[move] = 1
    return result
    

In [41]:
#function for converting 'promote' to formats the nn can use
#instead of storing 2 it stores [0,0,1] for example
def parsePromote(promoteTo):
    result = np.zeros(3,dtype = np.float32)
    result[int(promoteTo)] = 1
    return result

In [42]:
#split boards into chars
moveFrame['board'] = moveFrame['board'].apply(parseBoard)
promoteFrame['board'] = promoteFrame['board'].apply(parseBoard)

In [43]:
#convert moves
moveFrame['to'] = moveFrame['to'].apply(parseMove)
moveFrame['from'] = moveFrame['from'].apply(parseMove)

In [44]:
#convert promotions
promoteFrame['promote'] = promoteFrame['promote'].apply(parsePromote)

In [48]:
#view the first 5 lines of our final moveFrame
print(moveFrame.head())

                                                   board  \
26343  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
12945  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
17743  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
12259  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
21680  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                  ...   
33750  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3692   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
7705   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
13642  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
26500  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                    from  \
26343  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
12945  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
17743  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
12259  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [46]:
#view the first 5 lines of our final promoteFrame
print(promoteFrame.head())

                                                 board          promote
252  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  [0.0, 0.0, 1.0]
928  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  [1.0, 0.0, 0.0]
213  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  [0.0, 0.0, 1.0]
590  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  [0.0, 0.0, 1.0]
322  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  [0.0, 1.0, 0.0]


In [49]:
#manually fill arrays so we have the right shape, slow and stupid but it works
#keras kept complaining otherwise
moveFrameSize = moveFrame.shape[0]
promoteFrameSize = promoteFrame.shape[0]


moveInput = np.empty((moveFrameSize,512))
toOutput = np.empty((moveFrameSize,64))
fromOutput = np.empty((moveFrameSize,64))
promoteInput = np.empty((promoteFrameSize,512))
promoteOutput = np.empty((promoteFrameSize,3))


for i in range(moveFrameSize):
    array = np.asarray(moveFrame.loc[[i]]["board"])
    for j in range(512):
        moveInput[i][j] = array[0][j]
print("moveInput done")
        
for i in range(moveFrameSize):
    array = np.asarray(moveFrame.loc[[i]]["to"])
    for j in range(64):
        toOutput[i][j] = array[0][j]
print("toOutput done")

for i in range(moveFrameSize):
    array = np.asarray(moveFrame.loc[[i]]["from"])
    for j in range(64):
        fromOutput[i][j] = array[0][j]
print("fromOutput done")

for i in range(promoteFrameSize):
    array = np.asarray(promoteFrame.loc[[i]]["board"])
    for j in range(512):
        promoteInput[i][j] = array[0][j]
print("promoteInput done")

for i in range(promoteFrameSize):
    array = np.asarray(promoteFrame.loc[[i]]["promote"])
    for j in range(3):
        promoteOutput[i][j] = array[0][j]
print("promoteOutput done")


moveInput done
toOutput done
fromOutput done
promoteInput done
promoteOutput done


In [62]:
#initialize our models
toModel = keras.Sequential()
fromModel = keras.Sequential()
promoteModel = keras.Sequential()

In [63]:
#define the toModel
toModel.add(keras.layers.Dense(units=288,input_shape=(512,))) #hidden layer 1 with input
toModel.add(keras.layers.Dense(units=288)) #h2
toModel.add(keras.layers.Dense(units=64)) #output layer
toModel.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])#we will use adam and mse to train


#shape:
# 512 -> 288 -> 288 -> 64
# in     h1     h2     out

#use 288 because thats the mean of the neurons in the input and output layers.

In [64]:
#train it
toModel.fit(moveInput,toOutput,epochs = 100)

Epoch 1/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0109 - accuracy: 0.5692
Epoch 2/100
1155/1155 [==============================] - 9s 7ms/step - loss: 0.0099 - accuracy: 0.5976
Epoch 3/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0097 - accuracy: 0.5987
Epoch 4/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0097 - accuracy: 0.5982
Epoch 5/100
1155/1155 [==============================] - 7s 6ms/step - loss: 0.0097 - accuracy: 0.5988
Epoch 6/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0097 - accuracy: 0.6043
Epoch 7/100
1155/1155 [==============================] - 9s 7ms/step - loss: 0.0096 - accuracy: 0.6011
Epoch 8/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0096 - accuracy: 0.6041
Epoch 9/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0096 - accuracy: 0.6042
Epoch 10/100
1155/1155 [==============================] - 10s 8ms/step - 

1155/1155 [==============================] - 10s 8ms/step - loss: 0.0094 - accuracy: 0.6097
Epoch 80/100
1155/1155 [==============================] - 12s 10ms/step - loss: 0.0094 - accuracy: 0.6113
Epoch 81/100
1155/1155 [==============================] - 11s 9ms/step - loss: 0.0094 - accuracy: 0.6123
Epoch 82/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0094 - accuracy: 0.6123
Epoch 83/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0094 - accuracy: 0.6133
Epoch 84/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0094 - accuracy: 0.6124
Epoch 85/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0094 - accuracy: 0.6135
Epoch 86/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0094 - accuracy: 0.6115
Epoch 87/100
1155/1155 [==============================] - 9s 8ms/step - loss: 0.0094 - accuracy: 0.6118
Epoch 88/100
1155/1155 [==============================] - 9s 8ms/step - l

In [65]:
#define and train the from model
fromModel.add(keras.layers.Dense(units=288,input_shape=(512,))) #hidden layer 1 with input
fromModel.add(keras.layers.Dense(units=288)) #h2
fromModel.add(keras.layers.Dense(units=64)) #output layer
fromModel.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])#we will use sgd and mse to train
fromModel.fit(moveInput,fromOutput,epochs = 100)

Epoch 1/100
1155/1155 [==============================] - 10s 8ms/step - loss: 0.0145 - accuracy: 0.2743
Epoch 2/100
1155/1155 [==============================] - 10s 8ms/step - loss: 0.0135 - accuracy: 0.3055
Epoch 3/100
1155/1155 [==============================] - 11s 9ms/step - loss: 0.0134 - accuracy: 0.3125
Epoch 4/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0133 - accuracy: 0.3116
Epoch 5/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0133 - accuracy: 0.3129
Epoch 6/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0133 - accuracy: 0.3145
Epoch 7/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0133 - accuracy: 0.3194
Epoch 8/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0133 - accuracy: 0.3200
Epoch 9/100
1155/1155 [==============================] - 10s 9ms/step - loss: 0.0132 - accuracy: 0.3207
Epoch 10/100
1155/1155 [==============================] - 13s 11ms/st

1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3335
Epoch 80/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3335
Epoch 81/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3358
Epoch 82/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3335
Epoch 83/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3335
Epoch 84/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3337
Epoch 85/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3338
Epoch 86/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3343
Epoch 87/100
1155/1155 [==============================] - 8s 7ms/step - loss: 0.0131 - accuracy: 0.3345
Epoch 88/100
1155/1155 [==============================] - 8s 7ms/step - loss:

In [66]:
#define and train the promote model
promoteModel.add(keras.layers.Dense(units=100,input_shape=(512,))) #hidden layer 1 with input
promoteModel.add(keras.layers.Dense(units=100)) #h2
promoteModel.add(keras.layers.Dense(units=3)) #output layer
promoteModel.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])#we will use sgd and mse to train
promoteModel.fit(promoteInput,promoteOutput,epochs = 100)

Epoch 1/100
32/32 [==============================] - 0s 5ms/step - loss: 0.1528 - accuracy: 0.7572
Epoch 2/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0391 - accuracy: 0.9750
Epoch 3/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0264 - accuracy: 0.9810
Epoch 4/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0243 - accuracy: 0.9780
Epoch 5/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0232 - accuracy: 0.9830
Epoch 6/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0231 - accuracy: 0.9790
Epoch 7/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0217 - accuracy: 0.9780
Epoch 8/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0219 - accuracy: 0.9800
Epoch 9/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0221 - accuracy: 0.9770
Epoch 10/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0215 - accuracy: 0.9820
Epoch 11/

32/32 [==============================] - 0s 5ms/step - loss: 0.0195 - accuracy: 0.9820
Epoch 84/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0192 - accuracy: 0.9800
Epoch 85/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0194 - accuracy: 0.9820
Epoch 86/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0194 - accuracy: 0.9800
Epoch 87/100
32/32 [==============================] - 0s 5ms/step - loss: 0.0198 - accuracy: 0.9780
Epoch 88/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0196 - accuracy: 0.9780
Epoch 89/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0199 - accuracy: 0.9800
Epoch 90/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0195 - accuracy: 0.9820
Epoch 91/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0193 - accuracy: 0.9810
Epoch 92/100
32/32 [==============================] - 0s 6ms/step - loss: 0.0193 - accuracy: 0.9770
Epoch 93/100


In [67]:
#save the models
toModel.save("./toModel.h5")
fromModel.save("./fromModel.h5")
promoteModel.save("./promoteModel.h5")